In [1]:
# ================================
# Hull Tactical Market Prediction
# Optimized Baseline (Paraphrased)
# ================================

import os
import numpy as np
import pandas as pd
import polars as pl
from warnings import filterwarnings
from scipy.optimize import minimize, Bounds
from gc import collect

import kaggle_evaluation.default_inference_server

filterwarnings("ignore")

# -------------------------------
# Global constants
# -------------------------------
MIN_POSITION = 0
MAX_POSITION = 2


# -------------------------------
# Custom evaluation metric
# -------------------------------
class UserVisibleError(Exception):
    """Custom error for invalid predictions."""
    pass


def adjusted_sharpe(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    """
    Compute a Sharpe-like score with penalties for excess volatility and poor returns.
    """

    solution = solution.copy()
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_POSITION:
        raise UserVisibleError(f"Prediction above max limit {MAX_POSITION}")
    if solution['position'].min() < MIN_POSITION:
        raise UserVisibleError(f"Prediction below min limit {MIN_POSITION}")

    # Strategy returns
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )

    excess = solution['strategy_returns'] - solution['risk_free_rate']
    cum_excess = (1 + excess).prod()
    mean_excess = cum_excess ** (1 / len(solution)) - 1
    std_excess = solution['strategy_returns'].std()

    annual_days = 252
    if std_excess == 0:
        raise ZeroDivisionError
    sharpe = mean_excess / std_excess * np.sqrt(annual_days)
    strat_vol = float(std_excess * np.sqrt(annual_days) * 100)

    # Market comparison
    market_excess = solution['forward_returns'] - solution['risk_free_rate']
    market_cum = (1 + market_excess).prod()
    market_mean = market_cum ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()
    market_vol = float(market_std * np.sqrt(annual_days) * 100)

    # Penalties
    excess_vol_penalty = (
        1 + max(0, strat_vol / market_vol - 1.2) if market_vol > 0 else 1
    )
    return_gap = max(0, (market_mean - mean_excess) * 100 * annual_days)
    return_penalty = 1 + (return_gap ** 2) / 100

    score = sharpe / (excess_vol_penalty * return_penalty)
    return float(min(score, 1_000_000))


# -------------------------------
# Load training data
# -------------------------------
train = pd.read_csv(
    "/kaggle/input/hull-tactical-market-prediction/train.csv",
    index_col="date_id"
)

# -------------------------------
# Optimization objective
# -------------------------------
def objective(x):
    recent = train[-180:].copy()
    submission = pd.DataFrame({'prediction': x.clip(0, 2)}, index=recent.index)
    return -adjusted_sharpe(recent, submission)


# Initial guess + optimization
x0 = np.full(180, 0.05)
res = minimize(objective, x0, method="Powell", bounds=Bounds(0, 2), tol=1e-8)
print(res)

optimal_preds = res.x


# -------------------------------
# Prediction function for Kaggle server
# -------------------------------
counter = 0

def predict(batch: pl.DataFrame) -> float:
    global counter, optimal_preds
    value = np.float64(optimal_preds[counter])
    print(f"[{counter}] Prediction: {value:.8f}")
    counter += 1
    return value


# -------------------------------
# Run Kaggle evaluation server
# -------------------------------
server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    server.serve()
else:
    server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -17.396311156232123
       x: [ 9.850e-02  5.236e-02 ...  7.168e-02  5.402e-09]
     nit: 26
   direc: [[ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [-1.796e-02  4.041e-04 ...  1.325e-03  0.000e+00]]
    nfev: 144556
[0] Prediction: 0.09849934
[1] Prediction: 0.05235790
[2] Prediction: 0.00000001
[3] Prediction: 0.00000001
[4] Prediction: 0.00000001
[5] Prediction: 0.00000001
[6] Prediction: 0.00000001
[7] Prediction: 0.04648664
[8] Prediction: 0.10261887
[9] Prediction: 0.00000001
